In [69]:
import jieba.posseg as pseg
import torch
from torch.utils import data
import numpy as np

wordslist=['a','ad','an','c','d','f','i','m','n','nr','ns','nt','o','p','q','r','tg','t','u','v','vd','vn','w','x','y','z']

dim = len(wordslist)

In [70]:
def TextToVector(sentences):
    length = len(sentences)
    X = torch.zeros((length,dim),dtype=torch.float)

    cnt = 0
    for sentence in sentences:
        words = pseg.cut(sentence)
        for word, pos in words:
            if pos in wordslist:
                X[cnt][wordslist.index(pos)] += 1

        cnt += 1

    return X


In [71]:
import pandas as pd
df = pd.read_excel('年龄较大孩子答案.xlsx')
text_data = df['案件描述']  # 使用列标签
score_data = df['可信度得分']

features = TextToVector(text_data)
labels = torch.tensor(score_data,dtype=torch.float)
# print(features.shape)
# print(labels.shape)
# print(features[0])
# print(labels[0])

In [72]:
def load_array(data_arrays, batch_size, is_train=False):
    """构造⼀个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [73]:
batch_size = 5
data_iter = load_array((features, labels), batch_size)

In [74]:
from torch import nn
netA = nn.Sequential(nn.Linear(dim, 1))
lossA = nn.MSELoss()
trainerA = torch.optim.SGD(netA.parameters(), lr=0.03)

In [75]:
sum_epochs = 3
for epoch in range(sum_epochs):
    for X, y in data_iter:
        l = lossA(netA(X) ,y)
        trainerA.zero_grad()
        l.backward()
        trainerA.step()
    l = lossA(netA(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss nan
epoch 2, loss nan
epoch 3, loss nan


D:\lib\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\lib\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\lib\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([112])) that is different to the input size (torch.Size([112, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  re


下面开始构建B模型

In [68]:
import jieba.analyse
TopkeyCnt = 10

In [67]:
dfkey = pd.read_excel('关键词库.xlsx')
typecnt = 6
keywordlist = []
keywordlist.append(dfkey['家庭保护'])
keywordlist.append(dfkey['学校保护'])
keywordlist.append(dfkey['社会保护'])
keywordlist.append(dfkey['网络保护'])
keywordlist.append(dfkey['政府保护'])
keywordlist.append(dfkey['司法保护'])

In [ ]:
def TextToKeyVector(sentences):
    length = len(sentences)
    X = torch.zeros((length,typecnt),dtype=torch.float)

    cnt = 0
    for sentence in sentences:
        for word, _ in jieba.analyse.extract_tags(sentence, topK=TopkeyCnt):
            for i in range(typecnt):
                if word in keywordlist[i]:
                    X[cnt][i] += 1
        cnt += 1
    return X

In [ ]:
featureskey = TextToKeyVector(text_data)
labels = torch.tensor(score_data,dtype=torch.float)
data_iter_key = load_array((featureskey, labels), batch_size)

In [76]:
netB = nn.Sequential(nn.Linear(typecnt, 1))
lossB = nn.MSELoss()
trainerB = torch.optim.SGD(netB.parameters(), lr=0.03)

In [ ]:
sum_epochs_key = 3
for epoch in range(sum_epochs_key):
    for X, y in data_iter_key:
        l = lossB(netB(X) ,y)
        trainerB.zero_grad()
        l.backward()
        trainerB.step()
    l = lossB(netB(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')